# LAB | Abstractive Question Answering

Abstractive question-answering focuses on the generation of multi-sentence answers to open-ended questions. It usually works by searching massive document stores for relevant information and then using this information to synthetically generate answers. This notebook demonstrates how Pinecone helps you build an abstractive question-answering system. We need three main components:

- A vector index to store and run semantic search
- A retriever model for embedding context passages
- A generator model to generate answers

# Install Dependencies

i run these first cells, because i had some problems

In [1]:
import torch
from sentence_transformers import SentenceTransformer
# Set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# Load a small, lightweight model
retriever = SentenceTransformer("all-MiniLM-L6-v2", device=device)
print("Retriever loaded successfully on", device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Retriever loaded successfully on cuda


In [2]:
!pip install -qU datasets==2.16.1 pinecone-client==3.1.0 sentence-transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.0/211.0 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 153.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from datasets import load_dataset
# load the SQuAD dataset which contains Wikipedia contexts
# We will use streaming mode and shuffle it
wiki_data = load_dataset(
  'squad',
  split='train',
  streaming=True
).shuffle(seed=960)

README.md: 0.00B [00:00, ?B/s]

# Load and Prepare Dataset

Our source data will be taken from the Wiki Snippets dataset, which contains over 17 million passages from Wikipedia. But, since indexing the entire dataset may take some time, we will only utilize 50,000 passages in this demo that include "History" in the "section title" column. If you want, you may utilize the complete dataset. Pinecone vector database can effortlessly manage millions of documents for you.

We are loading the dataset in the streaming mode so that we don't have to wait for the whole dataset to download (which is over 9GB). Instead, we iteratively download records one at a time.

In [6]:
# show the contents of a single document in the dataset
next(iter(wiki_data))

{'id': '56bf8c8aa10cfb140055116f',
 'title': 'Beyoncé',
 'context': 'In July 2002, Beyoncé continued her acting career playing Foxxy Cleopatra alongside Mike Myers in the comedy film, Austin Powers in Goldmember, which spent its first weekend atop the US box office and grossed $73 million. Beyoncé released "Work It Out" as the lead single from its soundtrack album which entered the top ten in the UK, Norway, and Belgium. In 2003, Beyoncé starred opposite Cuba Gooding, Jr., in the musical comedy The Fighting Temptations as Lilly, a single mother whom Gooding\'s character falls in love with. The film received mixed reviews from critics but grossed $30 million in the U.S. Beyoncé released "Fighting Temptation" as the lead single from the film\'s soundtrack album, with Missy Elliott, MC Lyte, and Free which was also used to promote the film. Another of Beyoncé\'s contributions to the soundtrack, "Summertime", fared better on the US charts.',
 'question': "How did the critics view the movie

In [13]:
wiki_data.info.description

''

In [15]:
history = wiki_data.filter(lambda x: 'History' in x['title'])

Let's iterate through the dataset and apply our filter to select the 50,000 historical passages. We will extract `article_title`, `section_title` and `passage_text` from each document.

In [16]:
from tqdm.auto import tqdm  # progress bar

total_doc_count = 10000

counter = 0
docs = []
# iterate through the dataset and apply our filter
for d in tqdm(history, total=total_doc_count):
    # Extract the fields we need from SQuAD dataset
    extracted_doc = {
        "wiki_id": d.get("id"),  # SQuAD has an 'id' field
        "article_title": d.get("title"), # SQuAD has a 'title' field
        "section_title": "History", # We are filtering by history, so we can set this directly
        "passage_text": d.get("context"), # SQuAD provides the passage in 'context'
    }
    # Ensure 'context' is not None or empty for valid passages
    if extracted_doc["passage_text"]:
        docs.append(extracted_doc) # Add the extracted document to your list

    # increase the counter on every iteration
    counter += 1

    # Stop when we have enough documents
    if counter >= total_doc_count:
        break

print(f"Collected {len(docs)} documents.")
# Optional: print the first document to verify
if docs:
    print("First collected document:", docs[0])

  0%|          | 0/10000 [00:00<?, ?it/s]

Collected 698 documents.
First collected document: {'wiki_id': '5726ebefdd62a815002e9552', 'article_title': 'History_of_science', 'section_title': 'History', 'passage_text': 'The English word scientist is relatively recent—first coined by William Whewell in the 19th century. Previously, people investigating nature called themselves natural philosophers. While empirical investigations of the natural world have been described since classical antiquity (for example, by Thales, Aristotle, and others), and scientific methods have been employed since the Middle Ages (for example, by Ibn al-Haytham, and Roger Bacon), the dawn of modern science is often traced back to the early modern period and in particular to the scientific revolution that took place in 16th- and 17th-century Europe. Scientific methods are considered to be so fundamental to modern science that some consider earlier inquiries into nature to be pre-scientific. Traditionally, historians of science have defined science sufficie

In [10]:
#from tqdm.auto import tqdm  # progress bar

#total_doc_count = 50000

#counter = 0
#docs = []

#for d in tqdm(history, total=total_doc_count):

    #docs.append({
        #'article_title': d['article_title'],
        #'section_title': d['section_title'],
        #'passage_text': d['passage_text']
   # })
   # counter += 1
    #if counter >= total_doc_count:
    #    break




  0%|          | 0/50000 [00:00<?, ?it/s]

KeyError: 'section_title'

In [17]:
import pandas as pd

# create a pandas dataframe with the documents we extracted
df = pd.DataFrame(docs)
df.head()

,wiki_id,article_title,section_title,passage_text
0,5726ebefdd62a815002e9552,History_of_science,History,The English word scientist is relatively recen...
1,5726fa9cdd62a815002e96bd,History_of_science,History,The astronomer Aristarchus of Samos was the fi...
2,5726fa9cdd62a815002e96bf,History_of_science,History,The astronomer Aristarchus of Samos was the fi...
3,5726f1c9f1498d1400e8f0a9,History_of_science,History,Ancient Egypt made significant advances in ast...
4,5726f997f1498d1400e8f18b,History_of_science,History,The important legacy of this period included s...


In [18]:
df

,wiki_id,article_title,section_title,passage_text
0,5726ebefdd62a815002e9552,History_of_science,History,The English word scientist is relatively recen...
1,5726fa9cdd62a815002e96bd,History_of_science,History,The astronomer Aristarchus of Samos was the fi...
2,5726fa9cdd62a815002e96bf,History_of_science,History,The astronomer Aristarchus of Samos was the fi...
3,5726f1c9f1498d1400e8f0a9,History_of_science,History,Ancient Egypt made significant advances in ast...
4,5726f997f1498d1400e8f18b,History_of_science,History,The important legacy of this period included s...
...,...,...,...,...
693,5728bfa72ca10214002da6e3,History_of_India,History,"After 1857, the colonial government strengthen..."
694,5728a4d44b864d1900164b52,History_of_India,History,"In 1526, Babur, a Timurid descendant of Timur ..."
695,5728cc17ff5b5019007da6e2,History_of_India,History,"Bal Gangadhar Tilak, an Indian nationalist lea..."
696,5728a1f12ca10214002da4f0,History_of_India,History,For two and a half centuries from the mid 13th...


# Initialize Pinecone Index

The Pinecone index stores vector representations of our historical passages which we can retrieve later using another vector (query vector). To build our vector index, we must first establish a connection with Pinecone. For this, we need an API from Pinecone. You can get one for free from [here](https://app.pinecone.io/), and after that, we initialize the connection as follows:

In [19]:
#pip install pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 745.9/745.9 kB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.9/280.9 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0


In [24]:
from google.colab import userdata
import os

# Retrieve the API keys

PINECONE_API_KEY = userdata.get('PINECONE_APY_KEY')

# Set as environment variables

os.environ['PINECONE_APY_KEY'] = PINECONE_API_KEY


print("Pinecone API key loaded and set as environment variable.")

Pinecone API key loaded and set as environment variable.


In [ ]:
# CODE FROM PINECONE WEBSITEindex_name = "developer-quickstart-py"

# if not pc.has_index(index_name):
   #  pc.create_index_for_model(
    #     name=index_name,
    #     cloud="aws",
       #  region="us-east-1",
     #    embed={
     #        "model":"llama-text-embed-v2",
      #       "field_map":{"text": "chunk_text"}
   #      }
   #  )

In [ ]:
# import os
# from pinecone import Pinecone

# initialize connection to pinecone (get API key at app.pinecone.io)
# api_key = os.environ.get('PINECONE_API_KEY') or 'PINECONE_API_KEY'

# configure client
# pc = Pinecone(api_key=api_key)

Now we setup our index specification, this allows us to define the cloud provider and region where we want to deploy our index. You can find a list of all [available providers and regions here](https://docs.pinecone.io/docs/projects).

In [35]:
from pinecone import ServerlessSpec
import os

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

Now we create a new index. We will name it "abstractive-question-answering" — you can name it anything we want. We specify the metric type as "cosine" and dimension as 768 (384 BECAUSE I CHANGE TO LIGHT MODEL)  because the retriever we use to generate context embeddings is optimized for cosine similarity and outputs 384-dimension vectors.

In [39]:
index_name = "abstractive-question-answering" #give your index a meaningful name

In [40]:
import time

# check if index already exists (it shouldn't if this is first time)
 #initialize the index, and insure the stats are all zeros

# check if the extractive-question-answering index exists
if index_name not in pc.list_indexes().names():
    # create the index if it does not exist
    pc.create_index(
        name=index_name,
        dimension=384,  # dimensionality of all-MiniLM-L6-v2. I CHANGED THE MODEL: all-MiniLM-L6-v2
        metric='cosine',
        spec=spec
    )
# connect to extractive-question-answering index we created
index = pc.Index(index_name)

In [38]:
pc.delete_index(index_name)


# Initialize Retriever

Next, we need to initialize our retriever. The retriever will mainly do two things:

- Generate embeddings for all historical passages (context vectors/embeddings)
- Generate embeddings for our questions (query vector/embedding)

The retriever will create embeddings such that the questions and passages that hold the answers to our queries are close to one another in the vector space. We will use a SentenceTransformer model based on Microsoft's MPNet as our retriever. This model performs quite well for comparing the similarity between queries and documents. We can use Cosine Similarity to compute the similarity between query and context vectors generated by this model (Pinecone automatically does this for us).

# Generate Embeddings and Upsert

Next, we need to generate embeddings for the context passages. We will do this in batches to help us more quickly generate embeddings and upload them to the Pinecone index. When passing the documents to Pinecone, we need an id (a unique value), context embedding, and metadata for each document representing context passages in the dataset. The metadata is a dictionary containing data relevant to our embeddings, such as the article title, section title, passage text, etc.

In [41]:
# We will use batches of 64
batch_size = 64

print(f"\nStarting embedding generation and upsert for {len(df)} documents into Pinecone index '{index_name}'...")

# Iterate through the DataFrame in batches
for i in tqdm(range(0, len(df), batch_size), desc="Upserting batches to Pinecone"):
    # Find the end index for the current batch
    i_end = min(i + batch_size, len(df))

    # Extract the current batch of documents from the DataFrame
    batch_df = df.iloc[i:i_end]

    # Extract the 'passage_text' for embedding generation
    batch_texts = batch_df['passage_text'].tolist()

    # Generate embeddings for the current batch of texts
    batch_embeddings = retriever.encode(batch_texts, show_progress_bar=False, device=device).tolist()

    # Prepare records for Pinecone upsert
    vectors_for_batch = []
    for j, row in enumerate(batch_df.itertuples(index=False)):
        # The unique ID for each passage comes from the 'wiki_id' column
        doc_id = str(row.wiki_id) # Ensure the ID is a string

        # Create the metadata dictionary
        metadata = {
            'article_title': row.article_title,
            'section_title': row.section_title,
            'passage_text': row.passage_text, # Common to include the original text in metadata for retrieval
        }
        # Add any other relevant columns from your 'df' to metadata here if you wish

        vectors_for_batch.append({
            'id': doc_id,
            'values': batch_embeddings[j], # 'j' is the index within the current batch_embeddings list
            'metadata': metadata
        })

    # Upsert the prepared batch of vectors to the Pinecone index
    try:
        index.upsert(vectors=vectors_for_batch) # Corrected syntax here
    except Exception as e:
        print(f"\nError upserting batch {i}-{i_end}: {e}")
        # Consider logging the specific batch causing issues or implementing a retry mechanism

print("\nFinished generating embeddings and upserting all batches to Pinecone.")

# Final check: Describe index stats to verify the total number of vectors
print("\nFinal Pinecone Index Statistics:")
final_index_stats = index.describe_index_stats()
if '' in final_index_stats.namespaces:
    print(f"Total vectors in index: {final_index_stats.namespaces[''].vector_count}")
else:
    print(f"Total vectors in index: 0 (No default namespace found, likely empty index).")
print(final_index_stats)


Starting embedding generation and upsert for 698 documents into Pinecone index 'abstractive-question-answering'...


Upserting batches to Pinecone:   0%|          | 0/11 [00:00<?, ?it/s]


Finished generating embeddings and upserting all batches to Pinecone.

Final Pinecone Index Statistics:
Total vectors in index: 0 (No default namespace found, likely empty index).
{'_response_info': {'raw_headers': {'connection': 'keep-alive',
                                    'content-length': '185',
                                    'content-type': 'application/json',
                                    'date': 'Wed, 19 Nov 2025 16:33:40 GMT',
                                    'grpc-status': '0',
                                    'server': 'envoy',
                                    'x-envoy-upstream-service-time': '42',
                                    'x-pinecone-request-id': '8636684007512612685',
                                    'x-pinecone-request-latency-ms': '42'}},
 'dimension': 384,
 'index_fullness': 0.0,
 'memoryFullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'__default__': {'vector_count': 698}},
 'storageFullness': 0.0,
 'total_vector_count': 698,
 '

# Initialize Generator

We will use ELI5 BART for the generator which is a Sequence-To-Sequence model trained using the ‘Explain Like I’m 5’ (ELI5) dataset. Sequence-To-Sequence models can take a text sequence as input and produce a different text sequence as output.

The input to the ELI5 BART model is a single string which is a concatenation of the query and the relevant documents providing the context for the answer. The documents are separated by a special token &lt;P>, so the input string will look as follows:

>question: What is a sonic boom? context: &lt;P> A sonic boom is a sound associated with shock waves created when an object travels through the air faster than the speed of sound. &lt;P> Sonic booms generate enormous amounts of sound energy, sounding similar to an explosion or a thunderclap to the human ear. &lt;P> Sonic booms due to large supersonic aircraft can be particularly loud and startling, tend to awaken people, and may cause minor damage to some structures. This led to prohibition of routine supersonic flight overland.

More detail on how the ELI5 dataset was built is available [here](https://arxiv.org/abs/1907.09190) and how ELI5 BART model was trained is available [here](https://yjernite.github.io/lfqa.html).

Let's initialize the BART model using transformers.

In [29]:
from transformers import BartTokenizer, BartForConditionalGeneration

# load bart tokenizer and model from huggingface
tokenizer = BartTokenizer.from_pretrained('vblagoje/bart_lfqa')
generator = BartForConditionalGeneration.from_pretrained('vblagoje/bart_lfqa').to(device)

tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

All the components of our abstract QA system are complete and ready to be queried. But first, let's write some helper functions to retrieve context passages from Pinecone index and to format the query in the way the generator expects the input.

In [42]:
def query_pinecone(query, top_k):
    # generate embeddings for the query
    xq = retriever.encode([query]).tolist()
    # search pinecone index for context passage with the answer
    xc = xc = index.query(
        vector=xq,
        top_k=top_k,
        include_metadata=True) # I
    return xc

In [43]:
def format_query(query, context):
    # Extract passage_text from Pinecone search results and add the <P> tag
    context_texts = [f"<P> {m['metadata']['passage_text']}" for m in context]

    # Concatenate all context passages into a single string
    full_context = " ".join(context_texts)

    # Concatenate the query with the retrieved context
    formatted_query = f"Question: {query}\nContext: {full_context}"

    return formatted_query

Let's test the helper functions. We will query the Pinecone index function we created earlier with the `query_pinecone` to get context passages and pass them to the `format_query` function.

In [44]:
query = "when was the first electric power system built?"
result = query_pinecone(query, top_k=1)
result

QueryResponse(matches=[{'id': '5727f9a33acd2414000df132',
 'metadata': {'article_title': 'History_of_science',
              'passage_text': 'In 1687, Isaac Newton published the Principia '
                              'Mathematica, detailing two comprehensive and '
                              "successful physical theories: Newton's laws of "
                              'motion, which led to classical mechanics; and '
                              "Newton's Law of Gravitation, which describes "
                              'the fundamental force of gravity. The behavior '
                              'of electricity and magnetism was studied by '
                              'Faraday, Ohm, and others during the early 19th '
                              'century. These studies led to the unification '
                              'of the two phenomena into a single theory of '
                              'electromagnetism, by James Clerk Maxwell (known '
                    

In [45]:
from pprint import pprint

In [46]:
# format the query in the form generator expects the input
query = format_query(query, result["matches"])
pprint(query)

('Question: when was the first electric power system built?\n'
 'Context: <P> In 1687, Isaac Newton published the Principia Mathematica, '
 "detailing two comprehensive and successful physical theories: Newton's laws "
 "of motion, which led to classical mechanics; and Newton's Law of "
 'Gravitation, which describes the fundamental force of gravity. The behavior '
 'of electricity and magnetism was studied by Faraday, Ohm, and others during '
 'the early 19th century. These studies led to the unification of the two '
 'phenomena into a single theory of electromagnetism, by James Clerk Maxwell '
 "(known as Maxwell's equations).")


The output looks great. Now let's write a function to generate answers.

In [47]:
def generate_answer(query):
    # tokenize the query to get input_ids
    inputs = tokenizer([query], max_length=1024, return_tensors="pt").to(device)
    # use generator to predict output ids
    ids = generator.generate(inputs["input_ids"], num_beams=2, min_length=20, max_length=40)
    # use tokenizer to decode the output ids
    answer = tokenizer.batch_decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    return pprint(answer)

In [48]:
generate_answer(query)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


('The first electric power system was built in the early 19th century. The '
 'first electric power plant was built in the early 19th century. The first '
 'electric power plant was built in the early')


As we can see, the generator used the provided context to answer our question. Let's run some more queries.

In [49]:
query = "How was the first wireless message sent?"
context = query_pinecone(query, top_k=5)
query = format_query(query, context["matches"])
generate_answer(query)

("I'm not sure if this is what you're looking for, but I can tell you that the "
 'first wireless message was sent in the early 1900s. It was sent by a '
 'telegraph')


To confirm that this answer is correct, we can check the contexts used to generate the answer.

In [50]:
for doc in context["matches"]:
    print(doc["metadata"]["passage_text"], end='\n---\n')

In 1938 Otto Hahn and Fritz Strassmann discovered nuclear fission with radiochemical methods, and in 1939 Lise Meitner and Otto Robert Frisch wrote the first theoretical interpretation of the fission process, which was later improved by Niels Bohr and John A. Wheeler. Further developments took place during World War II, which led to the practical application of radar and the development and use of the atomic bomb. Though the process had begun with the invention of the cyclotron by Ernest O. Lawrence in the 1930s, physics in the postwar period entered into a phase of what historians have called "Big Science", requiring massive machines, budgets, and laboratories in order to test their theories and move into new frontiers. The primary patron of physics became state governments, who recognized that the support of "basic" research could often lead to technologies useful to both military and industrial applications. Currently, general relativity and quantum mechanics are inconsistent with e

In [ ]:
for doc in context["matches"]:
    print(doc["metadata"]["passage_text"], end='\n---\n')

by electrostatic induction or electromagnetic induction, which had too short a range to be practical. In 1866 Mahlon Loomis claimed to have transmitted an electrical signal through the atmosphere between two 600 foot wires held aloft by kites on mountaintops 14 miles apart. Thomas Edison had come close to discovering radio in 1875; he had generated and detected radio waves which he called "etheric currents" experimenting with high-voltage spark circuits, but due to lack of time did not pursue the matter. David Edward Hughes in 1879 had also stumbled on radio wave transmission which he received with his carbon microphone
---
the east coast of India, then on to Penang, Malacca, Singapore, Batavia (current Jakarta), to finally reach Darwin, Australia. It was the first direct link between Australia and Great Britain. The company that laid the first part of the cable took the name of Falmouth, Gibraltar and Malta Telegraph Company and had been founded in 1869. This company later operated as

In this case, the answer looks correct. If we ask a question and no relevant contexts are retrieved, the generator will typically return nonsensical or false answers, like with this question about COVID-19:

In [51]:
query = "where did COVID-19 originate?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context["matches"])
generate_answer(query)

('COVID-19 is a virus that causes the common cold. It is caused by a bacterium '
 'that infects the human body. It is not a virus that causes the common cold.')


In [ ]:
query = "where did COVID-19 originate?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context["matches"])
generate_answer(query)

('COVID-19 is a zoonotic disease, which means that it is a virus that is '
 'transmitted from one animal to another. It is not a virus that can be '
 'transmitted from person')


In [52]:
for doc in context["matches"]:
    print(doc["metadata"]["passage_text"], end='\n---\n')

In 1847, Hungarian physician Ignác Fülöp Semmelweis dramatically reduced the occurrency of puerperal fever by simply requiring physicians to wash their hands before attending to women in childbirth. This discovery predated the germ theory of disease. However, Semmelweis' findings were not appreciated by his contemporaries and came into use only with discoveries by British surgeon Joseph Lister, who in 1865 proved the principles of antisepsis. Lister's work was based on the important findings by French biologist Louis Pasteur. Pasteur was able to link microorganisms with disease, revolutionizing medicine. He also devised one of the most important methods in preventive medicine, when in 1880 he produced a vaccine against rabies. Pasteur invented the process of pasteurization, to help prevent the spread of disease through milk and other foods.
---
In 1847, Hungarian physician Ignác Fülöp Semmelweis dramatically reduced the occurrency of puerperal fever by simply requiring physicians to wa

In [ ]:
for doc in context["matches"]:
    print(doc["metadata"]["passage_text"], end='\n---\n')

to establish with certainty which diseases jumped from other animals to humans, but there is increasing evidence from DNA and RNA sequencing, that measles, smallpox, influenza, HIV, and diphtheria came to humans this way. Various forms of the common cold and tuberculosis also are adaptations of strains originating in other species.
Zoonoses are of interest because they are often previously unrecognized diseases or have increased virulence in populations lacking immunity. The West Nile virus appeared in the United States in 1999 in the New York City area, and moved through the country in the summer of 2002, causing much distress. Bubonic
---
plague is a zoonotic disease, as are salmonellosis, Rocky Mountain spotted fever, and Lyme disease.
A major factor contributing to the appearance of new zoonotic pathogens in human populations is increased contact between humans and wildlife. This can be caused either by encroachment of human activity into wilderness areas or by movement of wild ani

Let’s finish with a final few questions.

In [53]:
query = "what was the war of currents?"
context = query_pinecone(query, top_k=5)
query = format_query(query, context["matches"])
generate_answer(query)

('The war of currents is a term used to refer to a series of events that '
 'occurred in the early modern period. These events were characterized by a '
 'series of events known as the "Great Diver')


In [ ]:
query = "what was the war of currents?"
context = query_pinecone(query, top_k=5)
query = format_query(query, context["matches"])
generate_answer(query)

('The War of Currents was a series of events in the early 1900s between Edison '
 'and Westinghouse. The two companies were competing for the market share of '
 'electric power in the United States')


In [54]:
query = "who was the first person on the moon?"
context = query_pinecone(query, top_k=10)
query = format_query(query, context["matches"])
generate_answer(query)

('The first person to go to the moon was Apollo 11. The Apollo 11 astronauts '
 'landed on the moon in 1969. The Apollo 11 astronauts landed on the moon in '
 '1969. The Apollo 11 astronauts')


In [ ]:
query = "who was the first person on the moon?"
context = query_pinecone(query, top_k=10)
query = format_query(query, context["matches"])
generate_answer(query)

('The first person to walk on the moon was Neil Armstrong in 1969. He walked '
 'on the moon in 1969. He was the first person to walk on the moon.')


In [55]:
query = "what was NASAs most expensive project?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context["matches"])
generate_answer(query)

("I'm not sure if this counts as a project, but I think it's worth noting that "
 'the NASAs budget for the first year of its existence was about $1.5 billion.')


In [ ]:
query = "what was NASAs most expensive project?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context["matches"])
generate_answer(query)

('The Space Shuttle was the most expensive project in the history of NASA. It '
 'cost about $10 billion to build.')


As we can see, the model can generate some decent answers.

#### Add a few more questions

In [56]:
query = "what was the first nobel prize?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context["matches"])
generate_answer(query)

('The first Nobel Prize was awarded in 1884 by the American Academy of Arts '
 'and Sciences in recognition of the contributions of the American public to '
 'the study of science. The prize was given to the')


In [57]:
query = "what was the first country to reach 200 millions people?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context["matches"])
generate_answer(query)

('The first country to reach 200 million people was the Ottoman Empire. The '
 'Ottoman Empire was founded in the late 18th century, and the population of '
 'the Ottoman Empire peaked at about 200 million in')


In [58]:
query = "what is the large animal population?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context["matches"])
generate_answer(query)

("I'm not sure if this is what you're looking for, but I'll give it a shot. "
 "I'm a biologist, so I'll try to answer your question as best I can")


In [59]:
query = "what was the first country to send a person to the space?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context["matches"])
generate_answer(query)

('The first person to go to the moon was the first person to go to the moon. '
 'The first person to go to the moon was the first person to go to the moon. '
 'The first')


In [61]:
query = "what was the first person in the space with a rocket? and from what country?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context["matches"])
generate_answer(query)

('The first person in the space with a rocket was probably the first person in '
 'the space with a rocket. The first person in the space with a rocket was '
 'probably the first person in the space')
